# CMU auto-graded notebook

Before you turn these assignments in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE`, `<FILL IN>`, or "YOUR ANSWER HERE."


---


# CMU Machine Learning with Large Datasets

## Homework 1 - Coding 2: Streaming Naive Bayes


In [2]:
# Who did you collaborate with on this assignment?
# if no one, collaborators should contain an empty string,
# else list your collaborators below

collaborators = [""]
# YOUR CODE HERE
# raise NotImplementedError()

In [3]:
try:
    collaborators
except:
    raise AssertionError("you did not list your collaborators, if any")

### (1a) Environment Setup

Run the following cell to establish the runtime environment.


In [4]:
# Ignore this cell if the environment already has the packages

%pip install nose numpy


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
# imports that will be used in the notebook -- shouldn't need to import any other libraries

import math
import os
import re
from collections import Counter

import numpy as np
from nose.tools import assert_equal

### (1b) Data Preparation


We use the Reuters Corpus (RCV1), which is a set of news stories split into a hierarchy of categories. There are three file sets. The two data sets with "small" in the name contain smaller subsamples of the full data set. They are provided for debugging and local tests. The final classification task should use the "full" one. Each data set is split into a train and test set, as indicated by the file suffix:

- RCV1.full_train.txt
- RCV1.full_test.txt
- RCV1.small_train.txt
- RCV1.small_test.txt
- RCV1.very_small_train.txt
- RCV1.very_small_test.txt

Download the data using the link provided in the handout, and fill in the corresponding variables with their file paths in the following cell.


In [6]:
# TODO: Replace <FILL IN> with appropriate file paths
FULL_TRAIN = "/Users/leesunny/S25_10605/S25_HW1-hw1/10605_S25_HW1_RCV1/RCV1.full_train.txt"
FULL_TEST = "/Users/leesunny/S25_10605/S25_HW1-hw1/10605_S25_HW1_RCV1/RCV1.full_test.txt"
SMALL_TRAIN = "/Users/leesunny/S25_10605/S25_HW1-hw1/10605_S25_HW1_RCV1/RCV1.small_train.txt"
SMALL_TEST = "/Users/leesunny/S25_10605/S25_HW1-hw1/10605_S25_HW1_RCV1/RCV1.small_test.txt"
VERY_SMALL_TRAIN = "/Users/leesunny/S25_10605/S25_HW1-hw1/10605_S25_HW1_RCV1/RCV1.very_small_train.txt"
VERY_SMALL_TEST = "/Users/leesunny/S25_10605/S25_HW1-hw1/10605_S25_HW1_RCV1/RCV1.very_small_test.txt"

There are multiple class labels per document, meaning that there is more than one correct answer to the question "What kind of news article is this?"

For this assignment, we will ignore all class labels except for those ending in CAT and just be classifying into the top-level nodes of the hierarchy:

- CCAT: Corporate/Industrial
- ECAT: Economics
- GCAT: Government/Social
- MCAT: Markets

DO NOT change the following cell and just run it to set up the CAT labels


In [7]:
CAT_LABELS = ['CCAT', 'ECAT', 'GCAT', 'MCAT']

The data format is one document per line, with the class label(s) first (comma separated), a tab character, and then the document text.

Run the following cell to take a glance at the first document of the small training data set.


In [8]:
with open(SMALL_TRAIN, 'r') as f:
    print(f.readline())

C13,C21,CCAT	 The German government on Thursday awarded the first round of licences for basic public telephone services, opening the door for full competition to monopoly Deutsche Telekom. Kicking off the final stage of preparations ahead of liberalisation of European telecoms on January 1, 1998, Germany awarded licences for basic phone services to Vebacom GmbH, Britain's Colt Telecom Plc and NetCologne, a local operator.   The so-called &quot;class four&quot; licences cover basic voice telephone services for the public. That is the only service still under monopoly protection in Germany, where the telecoms market is expected to be worth more than 100 billion marks by the year 2000. In addition to the class 4 licences, the government awarded &quot;class 3&quot; licences to DBKom GmbH, the joint venture of a Mannesmann-led group and the German railway Deutsche Bahn AG, as well as to local operator VEW Telnet. Under a law passed this year, the class 3 licence allow owners of telecoms net

### (1c) Data Processing


To count the words, we need to tokenize the document text. In real-world practice, this may involve multiple steps, such as removing stop words and converting text to lowercase, which you learned in HW1: Entity Resolution. For this Naive Bayes part, we simplify the process by splitting only on words.

Run the following cell to define the `tokenization(doc)` function.


In [9]:
# DO NOT change this function
def tokenizeDoc(doc: str) -> list[str]:
    """
    Convert input document text into tokenization features
    Args:
        doc: document text
    Returns:
        list: a list of tokens
    """
    return re.findall('\\w+', doc)

As the handout instructs, streaming Naive Bayes loads one-line document at a time, use that document to update the classifier statistics, and then discard the document. After loading a line, we need a function to parse the line for classifier training.

Implement `parseDatafileLine(datafileLine)` that takes a line (labels + document text) and return a list of labels and a list of tokens. You need to use the `tokenizeDoc(doc)` function defined above.


In [10]:
def parseDatafileLine(datafileLine: str):
    """
    Parse a line of the data file to separate labels and document tokens
    Args:
        datafileLine: input string that is a line from the data file
    Returns:
        labels (list), tokens (list)
    """
    # TODO: YOUR CODE HERE
    # split the line into the labels and the document text using a tab
    parts = datafileLine.strip().split('\t')
    # the first part of the split is the labels, which are space-separated
    labels = parts[0].split(',')
    # the rest fo the line is the doc text
    document_text = parts[1] if len(parts)>1 else ''
    # tokenize the doc text
    tokens = tokenizeDoc(document_text)

    return labels, tokens
    # raise NotImplementedError()

In [11]:
"""Test parseDatafileLine(datafileLine)"""

line_sample1 = "C15,C151,CCAT\tMcDonald's Corp said Thursday it raised its quarterly dividend 10 percent, to $0.0825 a share from $0.075."
line_sample2 = "C15,C151,CCAT\tSix months to Sept 30, 1996       (in million rupees unless stated)"

assert_equal(parseDatafileLine(line_sample1),
             (['C15', 'C151', 'CCAT'],
              ['McDonald', 's', 'Corp', 'said', 'Thursday', 'it', 'raised', 'its',
               'quarterly', 'dividend', '10', 'percent', 'to', '0', '0825', 'a',
               'share', 'from', '0', '075']))

assert_equal(parseDatafileLine(line_sample2),
             (['C15', 'C151', 'CCAT'],
              ['Six', 'months', 'to', 'Sept', '30', '1996', 'in', 'million',
               'rupees', 'unless', 'stated']))

### 1(d) Training


Now, we are good for training. We will use a dictionary as the model to store the count statistics.

As the handout instructs, the model contains five parts:

- `y`: $(Y=y)$ for each label y the number of training instances of that class
- `ys`: $(Y=*)$ here $*$ means anything, so this is just the total number of training instances
- `y_w`: $(Y=y,W=w)$ number of times token w appears in a document with label y
- `y_ws`: $(Y=y,W=*)$ total number of tokens for documents with label y
- `vocabulary`: track the vocabulary for Laplace smoothing

Recall that Laplace smoothing formula is

$$p(W=w_i|Y=y)=\frac{count(Y=y,W=w_i) + \alpha}{count(Y=y,W=*)+ \alpha|V|}$$
where $|V|$ is the vocabulary.

Implement `nbmodel()` by figuring out the proper variable types for each part and filling in the blank below.


In [12]:
def nbmodel() -> dict:
    """
    Returns:
        dict storing the required five parts
    """
    # TODO: Replace <FILL IN> with appropriate code
    return {
        'y': {},
        'ys': 0,
        'y_w': {},
        'y_ws': {},
        'vocabulary': set()
    }

Implement `trainNB(trainfile, model)` that loads one document at a time and uses that document to update the required statistics for the Naive Bayes classifier.

Hint:

1. We only use those lables ending in CAT, which defined in `CAT_LABELS` earlier. Therefore, you need to figure out a way to skip other labels.
2. There are some documents with more than one CAT label. Treat those documents as multiple data instances (that is, add to the counters for
   all labels ending in CAT). For instance, if one line contains CCAT and ECAT, use the same document text twice.
3. It is not necessary to return the model here if you use proper types. Think about Python's mutable vs immutable types.


In [13]:
def trainNB(trainfile: str, model: dict):
    """
    Update the model in a streaming fashion.
    Args:
        trainfile: file path of the training data
        model: dict of the Naive Bayes classifier model
    """

    with open(trainfile, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            labels, tokens = parseDatafileLine(line)

            # Filter only labels ending in "CAT"
            relevant_labels = [label for label in labels if label.endswith("CAT")]

            # Update the model for each relevant label
            for label in relevant_labels:
                # Initialize structures for new labels
                if label not in model['y_w']:
                    model['y_w'][label] = {}

                # Update class count and total document count
                model['y'][label] = model['y'].get(label, 0) + 1
                model['ys'] += 1

                # Update token counts and vocabulary
                for token in tokens:
                    model['y_w'][label][token] = model['y_w'][label].get(token, 0) + 1
                    model['y_ws'][label] = model['y_ws'].get(label, 0) + 1
                    model['vocabulary'].add(token)

            # TODO: YOUR CODE HERE
            # raise NotImplementedError()

### 1(e) Test


For each line of documents, your classification code should get the best class $y$ and its log probabilities:

$$ln(p(Y=y))+\sum_{w_i} ln(p(W=w_i|Y=y))$$

Notice that we use the natural logarithm here.

Implement `testNB(testfile, model)` that uses the trained model to classify the test data and return a list of best classes, a list of max log probabilities (**rounding it to 4 decimal places**), and overall accuracy (**rounding it to 4 decimal places**). Please explicitly return in this specified order.


In [14]:
def testNB(testfile, model):
    """
    Implement Naive Bayes classification
    Args:
        testfile: file path of the test data
        model: dict of the Naive Bayes classifier model
    Returns:
        best_classes, log_probabilities, accuracy
    """
    best_classes = []
    log_probabilities = []
    correct_predictions = 0
    total_samples = 0

    # Get the size of the vocabulary for smoothing
    vocab_size = len(model['vocabulary'])

    with open(testfile, encoding='utf-8', errors='ignore') as f:
        for line in f.readlines():
            labels, tokens = parseDatafileLine(line)

            # initialize the dic to store log prob for each class
            class_log_probs = {}

            # calculate log prob for each class
            for label in model['y']:
                # log prior:
                log_prior = math.log((model['y'][label]) / sum(model['y'].values()))

                # log likelihood:
                log_likelihood = 0
                for token in tokens:
                    token_count = model['y_w'][label].get(token, 0)
                    token_prob = (token_count + 1) / (model['y_ws'][label] + vocab_size)
                    log_likelihood += math.log(token_prob)

                # total log prob for this class
                class_log_probs[label] = log_prior + log_likelihood

            # find the best class with highiest prob
            best_class = max(class_log_probs, key=class_log_probs.get)
            best_log_prob = class_log_probs[best_class]

            best_classes.append(best_class)
            log_probabilities.append(round(best_log_prob, 4))

            # check if the best class is among the true labels
            if best_class in labels:
                correct_predictions += 1

            total_samples += 1

    accuracy = round(correct_predictions / total_samples, 4)
            # raise NotImplementedError()
    return best_classes, log_probabilities, accuracy

In [15]:
"""DO NOT change this this cell and just run it to use the very small dataset to test your implementations"""

very_small_model = nbmodel()
trainNB(VERY_SMALL_TRAIN, very_small_model)
best_classes, log_probabilities, accuracy = testNB(VERY_SMALL_TEST, very_small_model)

assert_equal(best_classes,
             ['MCAT', 'ECAT', 'CCAT', 'ECAT', 'CCAT', 'CCAT', 'ECAT', 'CCAT'])
assert_equal(log_probabilities,
             [-9893.7804, -3912.8180, -1121.5992, -1610.1660,
              -701.3466, -1453.3430, -2218.3302, -2285.0698])
assert_equal(accuracy, 0.8750)

### 1(f) Full Classification and Deliverable

We are almost there! Let's wrap up this assignment.

Implement your training and test functions on the full dataset to get the full classification results. Please note that you need to define a new model different from the `very_small_model` we have already tested.

Write the full classification results to a file `full_result.txt` (please explicitly use this name). The output format should have one test result per line, and each line should have the format:

$$\text{[Label1, Label2, ...]<tab>Best class<tab>Log prob}$$

where **[Label1, Label2, ...]** are the true labels of the test instance, **Best class** is the class with the maximum log probability, and the last field is the log probability.

The last line of the file should include the accuracy.

Use the following cell to write your code.

Submit both this notebook and `full_result.txt` to Gradescope.


In [16]:
# * Here is an expected output of very_small_test dataset for your reference
# * You need to write one using the FULL dataset

'''
['C24', 'CCAT', 'M14', 'MCAT']\tMCAT\t-9893.7804
['E51', 'E512', 'ECAT', 'GCAT', 'GDIP']\tECAT\t-3912.8180
['C15', 'C152', 'C18', 'C181', 'CCAT']\tCCAT\t-1121.5992
['GCAT']\tECAT\t-1610.1660
['C13', 'CCAT', 'GCAT', 'GHEA']\tCCAT\t-701.3466
['C13', 'CCAT', 'M11', 'MCAT']\tCCAT\t-1453.3430
['C11', 'C13', 'CCAT', 'E12', 'ECAT', 'M13', 'M132', 'MCAT']\tECAT\t-2218.3302
['C31', 'CCAT']\tCCAT\t-2285.0698
Accuracy: 7/8=0.8750
'''

"\n['C24', 'CCAT', 'M14', 'MCAT']\tMCAT\t-9893.7804\n['E51', 'E512', 'ECAT', 'GCAT', 'GDIP']\tECAT\t-3912.8180\n['C15', 'C152', 'C18', 'C181', 'CCAT']\tCCAT\t-1121.5992\n['GCAT']\tECAT\t-1610.1660\n['C13', 'CCAT', 'GCAT', 'GHEA']\tCCAT\t-701.3466\n['C13', 'CCAT', 'M11', 'MCAT']\tCCAT\t-1453.3430\n['C11', 'C13', 'CCAT', 'E12', 'ECAT', 'M13', 'M132', 'MCAT']\tECAT\t-2218.3302\n['C31', 'CCAT']\tCCAT\t-2285.0698\nAccuracy: 7/8=0.8750\n"

In [17]:
def full_classification(train_file, test_file):
    """
    Train and test the Naive Bayes classifier on the full dataset and write results to a file.

    Args:
        train_file: File path to the training data.
        test_file: File path to the test data.
    """
    # Step 1: Initialize the Naive Bayes model
    full_model = nbmodel()

    # Step 2: Train the model on the full training dataset
    trainNB(train_file, full_model)

    # Step 3: Test the model on the full test dataset
    best_classes, log_probabilities, accuracy = testNB(test_file, full_model)

    # Step 4: Write the classification results to 'full_result.txt'
    with open("full_result.txt", "w") as result_file:
        total_samples = 0
        correct_predictions = 0

        # Read the test file and process each line
        with open(test_file, "r") as test_data:
            for line, best_class, log_prob in zip(test_data.readlines(), best_classes, log_probabilities):
                # Parse the true labels and tokens from the test file
                labels, _ = parseDatafileLine(line)
                total_samples += 1

                # Check if the prediction is correct
                if best_class in labels:
                    correct_predictions += 1

                # Write the result for this instance in the specified format
                result_file.write(f"[{', '.join(labels)}]\t{best_class}\t{log_prob:.4f}\n")

        # Step 5: Write the accuracy to the last line of the file
        accuracy_fraction = f"{correct_predictions}/{total_samples}={accuracy:.4f}"
        result_file.write(f"Accuracy: {accuracy_fraction}\n")

In [18]:
full_classification(FULL_TRAIN, FULL_TEST)